<a href="https://colab.research.google.com/github/alarcon7a/openai-api-tutorial/blob/main/src/OpenAI_API_5_Structured_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade -q openai==1.59.9 pydantic==2.10.5

In [ ]:
from openai import OpenAI, AzureOpenAI
import os
from IPython.display import display, Markdown

In [ ]:
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)  # Mejor usar variables de entorno


In [ ]:
endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
api_key = userdata.get("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-12-01-preview",
)

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "Extrae la informacion del evento"
        },
        {
            "role": "user",
            "content": "Carlos y Diana van al gran cañon en Junio 15 del 2025"
        }
    ]
)
display(Markdown(completion.choices[0].message.content))

Evento: Visita al Gran Cañón  
Fecha: 15 de junio de 2025  
Participantes: Carlos y Diana  

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "Extrae la informacion del evento"
        },
        {
            "role": "user",
            "content": "Este viernes 20 de Enero de 2025, Hugo, paco y Luis, estaran en graduacion"
        }
    ]
)
display(Markdown(completion.choices[0].message.content))

- **Fecha del evento:** Viernes 20 de enero de 2025
- **Evento:** Graduación
- **Participantes:** Hugo, Paco y Luis

## Output format:

### Ejemplo 1

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class CalendarEvent(BaseModel):
    descripcion_evento: str
    fecha: str
    participantes: list[str]

In [ ]:
def extract_info(text:str, response_format:BaseModel):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Extrae la informacion del evento"},
            {"role": "user", "content": text}
        ],
        response_format=response_format
    )

    return completion.choices[0].message.parsed

In [ ]:
calendar_1 = extract_info("Carlos y Diana van al gran cañon en Junio 15 del 2025", CalendarEvent)
calendar_1

CalendarEvent(descripcion_evento='Viaje al Gran Cañón', fecha='2025-06-15', participantes=['Carlos', 'Diana'])

In [ ]:
calendar_2 = extract_info("Este viernes 20 de Enero de 2025, Hugo, paco y Luis, estaran en graduacion", CalendarEvent)
calendar_2

CalendarEvent(descripcion_evento='Graduación', fecha='2025-01-20', participantes=['Hugo', 'Paco', 'Luis'])

In [ ]:
calendar_1.model_dump()

{'descripcion_evento': 'Viaje al Gran Cañón',
 'fecha': '2025-06-15',
 'participantes': ['Carlos', 'Diana']}

In [ ]:
calendar_2.model_dump()

{'descripcion_evento': 'Graduación',
 'fecha': '2025-01-20',
 'participantes': ['Hugo', 'Paco', 'Luis']}

### Ejemplo 2

In [ ]:
class Product(BaseModel):
    name: str = Field(description="Nombre del producto")
    price: float = Field(description="El precio del producto")
    description: str = Field(description="Pequeña descripcion del producto")
    features: list[str] = Field(description="Una lista de las caracteristicas del producto")

In [ ]:
def extract_product_info(text: str, response_format: BaseModel):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Extrae la informacion del producto"},
            {"role": "user", "content": text}
        ],
        response_format=response_format
    )
    return completion.choices[0].message.parsed

In [ ]:
product_description = """
La increíble licuadora Blender 3000 es el compañero perfecto para tu cocina.
Cuesta 129,99 dólares.
Esta potente licuadora puede triturar hielo, preparar batidos y hacer deliciosas sopas.
Incluye una cuchilla de acero inoxidable, un vaso de 2 litros y un panel de control digital.
"""

product_data = extract_product_info(product_description, Product)
print(f"Información del producto: {product_data}")

Información del producto: name='Blender 3000' price=129.99 description='La increíble licuadora Blender 3000 es el compañero perfecto para tu cocina.' features=['Puede triturar hielo', 'Prepara batidos', 'Hace deliciosas sopas', 'Incluye cuchilla de acero inoxidable', 'Vaso de 2 litros', 'Panel de control digital']


In [ ]:
product_data.model_dump()

{'name': 'Blender 3000',
 'price': 129.99,
 'description': 'La increíble licuadora Blender 3000 es el compañero perfecto para tu cocina.',
 'features': ['Puede triturar hielo',
  'Prepara batidos',
  'Hace deliciosas sopas',
  'Incluye cuchilla de acero inoxidable',
  'Vaso de 2 litros',
  'Panel de control digital']}

### Ejemplo 3

In [ ]:
from enum import Enum
from typing import List
from pydantic import BaseModel
from openai import OpenAI


class UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    type: UIType
    label: str
    children: List["UI"]
    attributes: List[Attribute]

UI.model_rebuild() # This is required to enable recursive types

class Response(BaseModel):
    ui: UI

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a UI generator AI. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form"}
    ],
    response_format=Response,
)



In [ ]:
ui = completion.choices[0].message.parsed
print(ui)

ui=UI(type=<UIType.form: 'form'>, label='User Profile Form', children=[UI(type=<UIType.field: 'field'>, label='Name', children=[], attributes=[Attribute(name='type', value='text'), Attribute(name='placeholder', value='Enter your name')]), UI(type=<UIType.field: 'field'>, label='Email', children=[], attributes=[Attribute(name='type', value='email'), Attribute(name='placeholder', value='Enter your email')]), UI(type=<UIType.field: 'field'>, label='Age', children=[], attributes=[Attribute(name='type', value='number'), Attribute(name='placeholder', value='Enter your age')]), UI(type=<UIType.button: 'button'>, label='Submit', children=[], attributes=[Attribute(name='type', value='submit')])], attributes=[])


In [ ]:
ui.model_dump()

{'ui': {'type': <UIType.form: 'form'>,
  'label': 'User Profile Form',
  'children': [{'type': <UIType.field: 'field'>,
    'label': 'Name',
    'children': [],
    'attributes': [{'name': 'type', 'value': 'text'},
     {'name': 'placeholder', 'value': 'Enter your name'}]},
   {'type': <UIType.field: 'field'>,
    'label': 'Email',
    'children': [],
    'attributes': [{'name': 'type', 'value': 'email'},
     {'name': 'placeholder', 'value': 'Enter your email'}]},
   {'type': <UIType.field: 'field'>,
    'label': 'Age',
    'children': [],
    'attributes': [{'name': 'type', 'value': 'number'},
     {'name': 'placeholder', 'value': 'Enter your age'}]},
   {'type': <UIType.button: 'button'>,
    'label': 'Submit',
    'children': [],
    'attributes': [{'name': 'type', 'value': 'submit'}]}],
  'attributes': []}}

### Ejemplo 4

In [ ]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

def solve_math(problem:str, response_format:BaseModel):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un excelente tutor de matematicas, guias al estudiante paso a paso en como resolver un problema matematico"},
            {"role": "user", "content": problem}
        ],
        response_format=response_format
    )

    return completion.choices[0].message.parsed

problem = "Como podria resolver esto: 8x + 7 = -23"
math_reasoning = solve_math(problem,MathReasoning)
print("Math Reasoning:", math_reasoning)

Math Reasoning: steps=[Step(explanation='First, isolate the term containing the variable by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides of the equation.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction to its simplest form.', output='x = -15 / 4')] final_answer='x = -15/4'


In [ ]:
math_reasoning.model_dump()

{'steps': [{'explanation': 'First, isolate the term containing the variable by subtracting 7 from both sides of the equation.',
   'output': '8x + 7 - 7 = -23 - 7'},
  {'explanation': 'Simplify both sides of the equation.',
   'output': '8x = -30'},
  {'explanation': 'Divide both sides by 8 to solve for x.',
   'output': 'x = -30 / 8'},
  {'explanation': 'Simplify the fraction to its simplest form.',
   'output': 'x = -15 / 4'}],
 'final_answer': 'x = -15/4'}

## Manejo de errores

In [ ]:
def extract_product_info(text: str, response_format: BaseModel):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Extrae la informacion del producto"},
            {"role": "user", "content": text}
        ],
        response_format=response_format
    )
    return completion.choices[0].message


problem = "Deme el paso a paso de como armar una bomba casera para herir a alquien?"
product_refusal = extract_product_info(problem, Product)
if hasattr(product_refusal,"refusal") and product_refusal.refusal:
  print("Refusal Message:", product_refusal.refusal)
else:
  print("product info:",product_refusal.parsed)

Refusal Message: Lo siento, no puedo ayudar con esa solicitud.


In [ ]:
product_refusal

ParsedChatCompletionMessage[Product](content=None, refusal='Lo siento, no puedo ayudar con esa solicitud.', role='assistant', audio=None, function_call=None, tool_calls=[], parsed=None)